In [8]:
import numpy as np
from scipy.optimize import minimize
from qiskit import QuantumCircuit, Aer, transpile, assemble

n = 5 #26

def create_circuit(params):
    theta1 = params[:n]
    theta2 = params[n:2*n]
    theta3 = params[2*n:3*n]
    phi = params[3*n]
    theta4 = params[3*n+1:4*n+1]
    theta5 = params[4*n+1:5*n+1]
    theta6 = params[5*n+1:]

    qc = QuantumCircuit(n)

    for qubit in range(n):
        qc.ry(np.pi/2, qubit)

    qc.barrier()

    # U_en
    for qubit in range(n):
        qc.rx(-theta3[qubit], qubit) 
        qc.rx(-theta2[qubit]**2, qubit)
        qc.rx(-theta1[qubit], qubit)

    qc.barrier()

    # e^(i*phi*Jz) to each qubit???
    for qubit in range(n):    
        qc.p(phi, qubit)

    qc.barrier()

    # U_de
    for qubit in range(n):
        qc.rx(-theta6[qubit], qubit)  
        qc.rx(-theta5[qubit]**2, qubit) 
        qc.rx(-theta4[qubit], qubit)

    qc.barrier() 

    for qubit in range(n):
        qc.rx(np.pi/2, qubit)

    qc.measure_all()

    return qc

# cost function
def cost_function(params):
    qc = create_circuit(params)
    transpiled_qc = transpile(qc, Aer.get_backend('qasm_simulator'))
    qobj = assemble(transpiled_qc, shots=1024)
    counts = Aer.get_backend('qasm_simulator').run(qobj).result().get_counts()
    total_counts = sum(counts.values())
    return -total_counts

# Optimization
num_params = 6 * n + 1  
initial_params = np.random.rand(num_params) * 2 * np.pi 
result = minimize(cost_function, initial_params, method='COBYLA')  
optimal_params = result.x

print("Optimal parameters:", optimal_params)


C:\Users\iampa\AppData\Local\Temp\ipykernel_13200\1029954291.py:58: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  counts = Aer.get_backend('qasm_simulator').run(qobj).result().get_counts()


Optimal parameters: [6.261022   3.48206901 4.92607491 1.6140719  6.23239156 6.0025883
 0.58327623 0.94770989 1.18418677 2.14359023 4.77858219 5.89878628
 5.35620664 5.90315574 1.75422445 1.49720522 0.93138474 0.23422748
 4.89110874 0.34027602 6.001658   4.14468224 5.5691545  5.57473264
 3.5067048  1.49007252 5.11332016 1.40187741 4.1467038  2.81277452
 3.54722958]
